<a href="https://colab.research.google.com/github/michelleaeh/CE888_Laboratory/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Lab 5**

 Load the data from "jester-data-1.csv"

In [0]:
import numpy as np
import pandas as pd

df=pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv")

Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
arr=df.values[:,1:]
rated=np.where(arr!=99)
def replace(orig, percentage=0.1):
  new_data=orig.copy()
  rated=np.where(arr!=99)
  n_rated=len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])
new_arr, idx = replace(arr, 0.1)

Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
n_latent_factors=2
user_ratings=arr
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))

In [0]:
def predict_rating(user_id,item_id):
  """ Predict a rating given a user_id and an item_id.
  """
  user_preference = latent_user_preferences[user_id]
  item_preference = latent_item_features[item_id]
  return user_preference.dot(item_preference)

In [0]:
def train(user_id, item_id, rating, alpha=0.0001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

In [0]:
def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if (iteration % 10) == 0:
            print(mse)

Calculate the performance (e.g., MSE) of the algorithm on the validation dataset

In [7]:
sgd()

26.21024572985242
23.054949137512466
20.95613521062119
18.19775910151857
17.32863043338752
17.18717071327152
17.148241575987022
17.136089565063287
17.132332840983356
17.131394948786927


In [11]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 2.9820594445786694)","(-0.29, 3.1480433820536757)","(6.36, 3.784715340286839)","(4.37, 5.265299173480569)","(-2.38, 2.208692434862938)","(-9.66, 2.519748517059752)","(-0.73, 2.961947705226993)","(-5.34, 2.3216765565319504)","(8.88, 5.071002110852039)","(9.22, 2.992309227034977)","(6.75, 2.3885685324185855)","(8.64, 2.561157862825146)","(4.42, 3.140481398703437)","(7.43, 2.4816729860948232)","(4.56, 3.0119594059174823)","(-0.97, 4.303513826233352)","(4.66, 2.4490076360401902)","(-0.68, 2.940929911970133)","(3.3, 2.716899426446998)","(-1.21, 3.576081338229908)","(0.87, 1.6232610131541516)","(8.64, 3.484837209223129)","(8.35, 4.0102731762873765)","(9.17, 5.512234115002663)","(0.05, 3.5344169402433203)","(7.57, 2.4248125924006327)","(4.71, 0.24407758011376499)","(0.87, 2.3923101941114)","(-0.39, 0.12760662522880997)","(6.99, 4.8667771045376025)","(6.5, 1.254446918474811)","(-0.92, 0.3299021175485151)","(7.14, 5.211099673407641)","(9.03, 2.7676975938253214)","(-1.8, 0.5293126566940493)","(0.73, 0.0561043675468328)","(7.09, 4.849785530180073)","(3.4, 3.0620544162867707)","(-0.87, 2.7867631614173773)","(7.91, 3.215811597458832)",...,"(-6.7, 1.3105401875511082)","(-3.35, 0.888720833731323)","(-9.03, 3.7869363229204875)","(4.47, 4.716441759917873)","(4.08, 1.1574770153530543)","(-3.83, 1.0779797209057833)","(8.74, 4.9492408405612975)","(1.12, 0.9528400930887864)","(0.78, 1.0180709858591663)","(7.52, 3.5324978818036077)","(-5.0, 3.977628093598914)","(2.77, 1.3017608612113616)","(8.3, 3.1071423983301583)","(7.77, 4.959023089729046)","(7.33, 3.7906764526421597)","(6.21, 1.4496691959917338)","(7.72, 3.330585533600719)","(8.98, 2.7245591510955878)","(8.64, 3.7943264577616778)","(8.2, 2.8861299734346)","(3.93, 2.234797727432089)","(4.85, 3.5260567237247735)","(4.85, 1.859096756937309)","(6.07, 3.5834278494959624)","(8.98, 3.411831040671494)","(4.51, 3.880693949150114)","(-0.05, 2.063897972892242)","(3.69, 2.1539461323336493)","(4.56, 0.41458615843731567)","(0.58, 3.791763166641253)","(2.82, 2.216692314084672)","(-4.95, 2.92898047740282)","(-0.29, 1.6057909555397087)","(7.86, 3.4781747406606933)","(-0.19, 3.0705077537743684)","(-2.14, 2.778372089663671)","(3.06, 2.4226143200814265)","(0.34, 2.514570054984155)","(-4.32, 3.9600731960150526)","(1.07, 2.7612111299832724)"
1,"(99.0, 6.878452616113779)","(99.0, 7.173566849529522)","(99.0, 7.474106877136841)","(99.0, 7.226568863001909)","(9.03, 5.50842464909638)","(9.27, 8.645699099978218)","(9.03, 4.946950289788303)","(9.27, 3.005642017013969)","(99.0, 7.607368579552719)","(99.0, 7.264342214862875)","(7.33, 8.358225451215613)","(7.57, 8.924688894487854)","(9.37, 2.8422963355421107)","(6.17, 8.856407433607323)","(-6.36, 2.130150552952042)","(-6.89, 1.8665582563259713)","(-7.86, 3.3614322710191313)","(9.03, 4.138584206292743)","(9.03, 5.8049282089042356)","(9.03, 5.6013328165464245)","(7.28, 8.482888508652016)","(99.0, 8.53154828570754)","(8.25, 7.702674574669269)","(99.0, 6.504615675531345)","(99.0, 8.146741045602042)","(7.48, 8.590442472862124)","(7.28, 7.500238198094253)","(7.28, 8.218845781049115)","(8.93, 7.077271456545001)","(99.0, 8.340011906088238)","(6.17, 8.008186289579015)","(7.28, 7.562865201273742)","(99.0, 7.181052577463806)","(99.0, 8.131531648611556)","(8.98, 7.791316842975015)","(7.33, 7.07577845035935)","(99.0, 6.040724818916233)","(6.17, 8.096554411982636)","(9.08, 8.56980418251887)","(7.33, 8.43747312726557)",...,"(6.46, 8.437902152673754)","(7.28, 8.114037923833223)","(99.0, 7.91249773038502)","(99.0, 6.902752974120535)","(7.04, 7.980912864509534)","(7.28, 7.930898436146983)","(99.0, 7.116997913582674)","(7.28, 7.730713033332549)","(8.25, 8.090515840048129)","(99.0, 8.088460284424771)","(99.0, 6.201764747044673)","(99.0, 8.312590056236258)

Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)

In [15]:
n_latent_factors=2
user_ratings=arr
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))
def predict_rating(user_id,item_id):
  """ Predict a rating given a user_id and an item_id.
  """
  user_preference = latent_user_preferences[user_id]
  item_preference = latent_item_features[item_id]
  return user_preference.dot(item_preference)
def train(user_id, item_id, rating, alpha=0.000001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if (iteration % 10) == 0:
            print(mse)
sgd()
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

27.662728175458142
27.13894590053896
26.73324950717206
26.417030496265614


KeyboardInterrupt: ignored

*OBSERVATIONS*
*   Increasing alpha to 0.01 returns nan MSE 
*   Reducing alpha to 0.000001 returns larger MSE
*   Increasing the number of latent factors to 7 converges the MSE at the same 5th value





Once you're happy, make predictions for the test dataset

(If you have time) Use pandas to find the best- and the worst-rated jokes